In [350]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer


In [351]:
positiveReviews = BeautifulSoup(open("positive.review").read())
positiveReviews = positiveReviews.findAll("review_text")


In [352]:
negativeReviews = BeautifulSoup(open("negative.review").read())
negativeReviews = negativeReviews.findAll("review_text")


In [353]:
np.random.shuffle(positiveReviews)

In [354]:
positiveReviews = positiveReviews[:len(positiveReviews)]
positiveReviewsDF = pd.DataFrame(positiveReviews, columns=["text"])
positiveReviewsDF["label"] = 1

In [355]:
negativeReviewsDF = pd.DataFrame(negativeReviews,columns=["text"])
negativeReviewsDF["label"] = 0


In [356]:
df = pd.concat([positiveReviewsDF,negativeReviewsDF])


In [357]:
def regexReplace(row):
   # text = re.sub("<review_text>","",row)
   # print(text)
    return row.text

In [358]:
X = df.text
y = df.label

In [345]:
X = X.apply(regexReplace)

In [346]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [347]:
# settings that you use for count vectorizer will go here
tfidf_vectorizer=TfidfVectorizer(use_idf=True,smooth_idf=True)
 
# just send in all your docs here
#X_train=tfidf_vectorizer.fit_transform(X_train)
#X_test = tfidf_vectorizer.transform(X_test)

In [349]:
cv = CountVectorizer(binary=True)
X_train = cv.fit_transform(X_train)
X_test = cv.transform(X_test)

AttributeError: lower not found

In [333]:
print(cv.get_feature_names())
word_list = cv.get_feature_names();    
count_list = X_train.toarray().sum(axis=0)
print(count_list)

['00', '000', '01', '0183', '04', '05', '06', '09', '0_20', '0gb', '10', '100', '1000', '1000mah', '100g', '101', '101n', '1024', '1030', '1050', '106', '108', '108g', '108mbps', '10d', '10in', '10k', '10min', '10s', '10x', '11', '110', '1100', '1100mah', '114', '11a', '11b', '11c', '11g', '11mb', '11mbps', '12', '120', '1200', '1200x1600p', '124', '125', '1250', '127', '128', '1280', '128kbps', '128mb', '12v', '13', '1311', '134', '13x19', '14', '140', '142', '1440', '149', '1499', '14ms', '15', '150', '1500', '152', '154', '15ft', '16', '160', '1600', '1600dpi', '1600mah', '160gb', '161', '1673s', '168', '1680', '1680x1050', '16mb', '16x', '16x9', '17', '1750', '18', '180', '18hz', '19', '190t', '192', '1920', '192k', '192kbps', '1970', '1986', '1988', '1989', '1990', '1992', '1g', '1gb', '1m', '1s', '1st', '20', '200', '2000', '2002', '2003', '2004', '2005', '2006', '200mhz', '2070n', '20d', '20gb', '20khz', '20th', '21', '212', '22', '2200', '2255', '22mm', '23', '230', '24', '240'

In [334]:
bag_of_words = cv.transform(X_train)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

AttributeError: lower not found

In [335]:
X_train

<1200x8747 sparse matrix of type '<class 'numpy.int64'>'
	with 81326 stored elements in Compressed Sparse Row format>

In [322]:
#use multinomial naive bayes
model = MultinomialNB()

model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(model.score(X_test,y_test))
print(classification_report(y_test, y_pred))

0.84125
              precision    recall  f1-score   support

           0       0.81      0.88      0.85       397
           1       0.87      0.80      0.84       403

   micro avg       0.84      0.84      0.84       800
   macro avg       0.84      0.84      0.84       800
weighted avg       0.84      0.84      0.84       800



In [323]:
#use adaboost
adaBoostModel = AdaBoostClassifier()
adaBoostModel.fit(X_train,y_train)
y_pred = adaBoostModel.predict(X_test)
print(adaBoostModel.score(X_test,y_test))
print(classification_report(y_test, y_pred))

0.78375
              precision    recall  f1-score   support

           0       0.81      0.73      0.77       397
           1       0.76      0.84      0.80       403

   micro avg       0.78      0.78      0.78       800
   macro avg       0.79      0.78      0.78       800
weighted avg       0.79      0.78      0.78       800



In [324]:
#use svm
randomForestClassifier = RandomForestClassifier()
randomForestClassifier.fit(X_train,y_train)
y_pred = randomForestClassifier.predict(X_test)
print(randomForestClassifier.score(X_test,y_test))
print(classification_report(y_test, y_pred))

0.71125
              precision    recall  f1-score   support

           0       0.68      0.79      0.73       397
           1       0.75      0.64      0.69       403

   micro avg       0.71      0.71      0.71       800
   macro avg       0.72      0.71      0.71       800
weighted avg       0.72      0.71      0.71       800



/root/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [365]:
from sklearn.linear_model import LogisticRegression

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    print ("Accuracy for C=%s: %s" 
           % (c, classification_report(y_test,y_pred)))
    print("score",LogisticRegression.score(X_test,y_test))

Accuracy for C=0.01:               precision    recall  f1-score   support

           0       0.63      0.96      0.76       397
           1       0.93      0.44      0.60       403

   micro avg       0.70      0.70      0.70       800
   macro avg       0.78      0.70      0.68       800
weighted avg       0.78      0.70      0.68       800

Accuracy for C=0.05:               precision    recall  f1-score   support

           0       0.73      0.89      0.80       397
           1       0.86      0.68      0.76       403

   micro avg       0.79      0.79      0.79       800
   macro avg       0.80      0.79      0.78       800
weighted avg       0.80      0.79      0.78       800

Accuracy for C=0.25:               precision    recall  f1-score   support

           0       0.77      0.85      0.81       397
           1       0.83      0.75      0.79       403

   micro avg       0.80      0.80      0.80       800
   macro avg       0.80      0.80      0.80       800
weighted av

In [368]:
# Create the parameter grid based on the results of random search 
# GridSearchCV to find optimal n_estimators
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [4,8,10,12,15,20],
    'min_samples_leaf': range(100, 900, 100),
    'min_samples_split': range(200, 700, 200),
    'n_estimators': [100,200, 300,500,1500], 
    'max_features': [5, 10,20,25,30]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                           n_jobs = -1,verbose = 1)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)

/root/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 3600 candidates, totalling 10800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   53.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed: 18.3min


In [ ]:
# printing the optimal accuracy score and hyperparameters
print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

In [ ]:
# model with the best hyperparameters
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(bootstrap=True,
                             max_depth=4,
                             min_samples_leaf=100, 
                             min_samples_split=200,
                             max_features=20,
                             n_estimators=200)